In [1]:
import pandas as pd
import numpy as np


def calculate_bearings_vectorized(df):
    # 将经纬度从度数转换为弧度
    lat_rad = np.radians(df["latitude"].values)
    lon_rad = np.radians(df["longitude"].values)

    # 使用NumPy的广播初始化经度差
    delta_lon = lon_rad[:, np.newaxis] - lon_rad

    # 计算所有成对的方位角
    x = np.sin(delta_lon) * np.cos(lat_rad[:, np.newaxis])
    y = np.cos(lat_rad) * np.sin(lat_rad[:, np.newaxis]) - np.sin(
        lat_rad) * np.cos(lat_rad[:, np.newaxis]) * np.cos(delta_lon)

    # np.arctan2返回值介于 [-π, π]之间，将其转换为[0, 2π]范围内
    initial_bearing = np.arctan2(x, y)
    initial_bearing = np.degrees(initial_bearing)
    bearing = (initial_bearing + 360) % 360

    # 设置自身到自身的方位角为NaN
    np.fill_diagonal(bearing, np.nan)

    return bearing


# 读取CSV文件
df = pd.read_csv("/home/yuxuan/wqy/Air_inference/data_/1085_stations.csv")

# 计算方位角
bearings = calculate_bearings_vectorized(df)

In [2]:
print(bearings)
#

[[         nan 357.34463592   0.38224859 ...  13.37969205  13.1400336
   12.50623281]
 [177.33776672          nan   5.77409269 ...  13.53286659  13.29110698
   12.66046728]
 [180.3837917  185.78248245          nan ...  13.58321587  13.34015367
   12.70854303]
 ...
 [196.69704804 196.84539782 196.88355783 ...          nan 268.13383346
  209.09142306]
 [196.39452481 196.54096578 196.57793187 ...  88.09420144          nan
  205.40864641]
 [195.45723963 195.60816938 195.64472379 ...  28.81373047  25.17209538
           nan]]


In [3]:
bearings = np.transpose(bearings)
print(bearings)

[[         nan 177.33776672 180.3837917  ... 196.69704804 196.39452481
  195.45723963]
 [357.34463592          nan 185.78248245 ... 196.84539782 196.54096578
  195.60816938]
 [  0.38224859   5.77409269          nan ... 196.88355783 196.57793187
  195.64472379]
 ...
 [ 13.37969205  13.53286659  13.58321587 ...          nan  88.09420144
   28.81373047]
 [ 13.1400336   13.29110698  13.34015367 ... 268.13383346          nan
   25.17209538]
 [ 12.50623281  12.66046728  12.70854303 ... 209.09142306 205.40864641
           nan]]


In [4]:
# 初始化8个区间数组
intervals = [np.zeros_like(bearings, dtype=np.int8) for _ in range(8)]


# 定义一个函数来检查角度是否属于特定区间
def in_interval(angle, interval_index):
    lower_bound = interval_index * (np.pi / 4)
    upper_bound = (interval_index + 1) * ((np.pi / 4))
    return (angle >= lower_bound) & (angle < upper_bound)


# 填充8个区间数组
for i in range(8):
    intervals[i] = in_interval(bearings, i).astype(np.int8)

In [5]:
print(intervals[0].shape)
print(intervals[0][:10])

(1085, 1085)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [6]:
for i in range(8):
    # 提取对角线元素
    diagonal_elements = np.diag(intervals[i])

    # 检查对角线元素是否全为0
    all_zeros = np.all(diagonal_elements == 0)

    # print("对角线元素:", diagonal_elements)
    print("对角线元素是否全为0:", all_zeros)

对角线元素是否全为0: True
对角线元素是否全为0: True
对角线元素是否全为0: True
对角线元素是否全为0: True
对角线元素是否全为0: True
对角线元素是否全为0: True
对角线元素是否全为0: True
对角线元素是否全为0: True


In [7]:
for i in range(8):
    # 统计矩阵中元素为1的数量
    count_ones = np.sum(intervals[i] == 1)

    print(f"矩阵intervals{i}中为1的元素数量:", count_ones)

矩阵intervals0中为1的元素数量: 2537
矩阵intervals1中为1的元素数量: 2604
矩阵intervals2中为1的元素数量: 2419
矩阵intervals3中为1的元素数量: 2584
矩阵intervals4中为1的元素数量: 2739
矩阵intervals5中为1的元素数量: 2863
矩阵intervals6中为1的元素数量: 3012
矩阵intervals7中为1的元素数量: 3138


In [8]:
np.save("/home/yuxuan/lgf/AirFormer/data/stations/intervals_all.npy",
        intervals)

In [10]:
np.savez_compressed(
    "/home/yuxuan/lgf/AirFormer/data/stations/bearing.npz",
    bearing0=intervals[0],
    bearing1=intervals[1],
    bearing2=intervals[2],
    bearing3=intervals[3],
    bearing4=intervals[4],
    bearing5=intervals[5],
    bearing6=intervals[6],
    bearing7=intervals[7],
)

In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# 从 CSV 文件中读取监测站点的经纬度数据
df = pd.read_csv("/home/yuxuan/wqy/Air_inference/data_/1085_stations.csv")

# 获取经纬度坐标列的数据
coordinates = df[["latitude", "longitude"]].values

# 创建一个 1085*1085 的 numpy 数组，用于存储距离
num_stations = len(coordinates)
distances = np.zeros((num_stations, num_stations))

# 计算每对监测站点之间的距离
for i in range(num_stations):
    for j in range(num_stations):
        # 获取当前两个监测站点的经纬度坐标
        coord1 = coordinates[i]
        coord2 = coordinates[j]

        # 计算两点之间的距离，并将结果存储在数组中
        distances[i][j] = geodesic(coord1, coord2).kilometers

# distances 数组现在包含了每对监测站点之间的距离


In [2]:
print(distances)

[[   0.           19.59334256   30.59614387 ... 2507.79907013
  2504.28133999 2329.87192854]
 [  19.59334256    0.           11.08011474 ... 2489.33095544
  2485.77868926 2311.26610628]
 [  30.59614387   11.08011474    0.         ... 2478.45944974
  2474.89590222 2300.35072648]
 ...
 [2507.79907013 2489.33095544 2478.45944974 ...    0.
    13.41242861  185.16812344]
 [2504.28133999 2485.77868926 2474.89590222 ...   13.41242861
     0.          178.66079655]
 [2329.87192854 2311.26610628 2300.35072648 ...  185.16812344
   178.66079655    0.        ]]


In [ ]:
# 您可以将它保存到文件中
np.save("/home/yuxuan/lgf/AirFormer/data/stations/distances.npy", distances)